In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.utils as vutils
import matplotlib.pyplot as plt

from discriminator import Discriminator
from generator import Generator

from torch.utils.data import DataLoader, Dataset


from utils import *
import torchvision

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:


def visualize(dataloader, netG, netD, criterion, epoch, fixed_noise, device):
    real_batch = next(iter(dataloader))
    real_batch = real_batch[0].to(device)
    y_labels = real_batch[1]
    # cast to int
    y_labels = y_labels.type(torch.LongTensor).to(device)
    batch_size = real_batch.size(0)
    label = torch.full((batch_size,), 1.0, device=device)
    output = netD(real_batch, y_labels).view(-1)
    errD_real = criterion(output, label)
    D_x = output.mean().item()

    fake = netG(fixed_noise, y_labels)
    label.fill_(0.0)
    output = netD(fake.detach()).view(-1)
    errD_fake = criterion(output, label)
    D_G_z1 = output.mean().item()

    errD = errD_real + errD_fake

    label.fill_(1.0)
    output = netD(fake).view(-1)
    errG = criterion(output, label)
    D_G_z2 = output.mean().item()

    print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
          % (epoch, 100, 0, len(dataloader),
             errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

    

    fake = netG(fixed_noise, y_labels)
    

    # Plot the fake images from the last epoch
    plt.figure(figsize=(8,8))
    plt.title("Fake Images")
    for row in range(8):
        for col in range(8):
            plt.subplot(8, 8, row*8+col+1)
            plt.axis("off")
            # set subplot label to be the label of the image
            plt.title(y_labels[row*8+col].item())
            plt.imshow(fake[row*8+col].detach().cpu().squeeze(), cmap='gray')


    # Plot the real images from the last epoch
    plt.figure(figsize=(8,8))
    plt.title("Real Images")
    for row in range(8):
        for col in range(8):
            plt.subplot(8, 8, row*8+col+1)
            plt.axis("off")
            plt.imshow(real_batch[row*8+col].detach().cpu().squeeze(), cmap='gray')




In [3]:
modelG = Generator(32, 1)
modelD = Discriminator()

checkpointG = torch.load('/ssd_scratch/cvit/anirudhkaushik/checkpoints/cganG_checkpoint_latest.pt')
checkpointD = torch.load('/ssd_scratch/cvit/anirudhkaushik/checkpoints/cganD_checkpoint_latest.pt')
modelG.load_state_dict(checkpointG['model'])
modelD.load_state_dict(checkpointD['model'])

modelG = modelG.to(device)
modelD = modelD.to(device)


fixed_noise = torch.randn(64, 100, 1, 1, device=device)

data_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32,32)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,))
])
train = torchvision.datasets.MNIST(root='/ssd_scratch/cvit/anirudhkaushik/datasets/', train=True, download=True, transform=data_transforms)
test = torchvision.datasets.MNIST(root='/ssd_scratch/cvit/anirudhkaushik/datasets/', train=False, download=True, transform=data_transforms)

data_loader = DataLoader(torch.utils.data.ConcatDataset([train, test]), batch_size=64, shuffle=True)


# visualize


In [4]:
criterion = nn.BCELoss()
visualize(data_loader, modelG, modelD, criterion, 0, fixed_noise, device)

/opt/conda/conda-bld/pytorch_1666642969563/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [238,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1666642969563/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [238,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1666642969563/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [238,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1666642969563/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [238,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1666642969563/work/aten/src/ATen/native/cuda/Indexing.cu:1141: indexSelectLargeIndex: block: [238,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`